In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import scipy.stats as st
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import pandas as pd
from io import StringIO
%matplotlib inline
import time

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import os

from sklearn.metrics import confusion_matrix
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from tqdm import tqdm

# New Cerebral Cortices

## Import data

In [ ]:
# list all csv file in folder
from os import listdir
 
def list_files(directory, extension):
    return (f for f in listdir(directory) if f.endswith('.' + extension))

directory = '/content/drive/MyDrive/Data for supervised analysis/cerebral_cortices_new/'
# directory = "D:/BOX/Box Sync/RA/data/IMAGING/DESI/from olof/shared-files-with-Olof/paper2/data/csv/"
# directory = "D:/BOX/Box Sync/RA/data/IMAGING/DESI/from olof/shared-files-with-Olof/paper1/normalcancer/extra/"

files = list_files(directory, "csv")
filenames=[]
for f in files:
    filenames.append(f)

print(len(filenames))    
filenames

In [ ]:
df0 = filenames[0]
df1 = filenames[1]
df2 = filenames[2]
df3 = filenames[3]


file0 = pd.read_csv(directory+df0) # change this directory to your folder
file1 = pd.read_csv(directory+df1)
file2 = pd.read_csv(directory+df2)
file3 = pd.read_csv(directory+df3)


file = pd.concat([file0, file1, file2, file3], ignore_index=True)
file.reset_index(drop = True)
Data = file.iloc[:,2:]

Data=Data.fillna(0) #replace NAs with 0
file

# Mean spec

In [ ]:
mean_spec = Data.mean()

In [ ]:
np.array(mean_spec)

In [ ]:
peaks = list(Data.columns)
results = list(map(float, peaks))

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi = 100)
plt.ylim(0,30000)
plt.xlabel('m/z')
plt.title('Mean spectrum with original data')
plt.plot(results, mean_spec)

In [ ]:
file['Label'].value_counts()

In [ ]:
file['health'] = file['ID']
for i in range(file.shape[0]):
    if file['Label'][i] == 'CC1':
        file['health'][i] = 1
    if file['Label'][i] == 'CC2':
        file['health'][i] = 1
    if file['Label'][i] == 'CC3':
        file['health'][i] = 0
    if file['Label'][i] == 'CC4':
        file['health'][i] = 0


In [ ]:
file['health'].value_counts()

## Define m/z axis

In [ ]:
tmp = Data.columns.values 
mz = tmp.astype(np.float)
mz_trans=mz.transpose()
mz.shape

## Log transform

In [ ]:
data = Data
# data = Data_r
# data = Data_f
tmp_log = data[data != 0]
logOS = np.nanmedian(tmp_log)
Data_log = np.log(data+logOS)
Data_log.shape

In [ ]:
logOS

## Normal PCA

In [ ]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3) #keep first 3 components
X_2D = pca.fit(Data_log).transform(Data_log)
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2','PC3'])
PCA_data=file
PCA_data['PC1'] = X_2D[:,0]
PCA_data['PC2'] = X_2D[:,1]
group_label = list(PCA_data.columns)
# PCA_data
colors=['magenta','green','navy', 'r', 'b', 'y', 'orange', 'indigo']
sns.lmplot("PC1", "PC2", hue=group_label[-3], data=PCA_data, palette=colors, fit_reg=False)

In [ ]:
#PCA cont.
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]

fig,axs = plt.subplots(ncols=3,constrained_layout=True,figsize=(15,5))

axs[0].bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
axs[0].set_ylabel('Percentage of Explained Variance')
axs[0].set_xlabel('Principal Component')
axs[0].set_title('Scree Plot')

axs[1].scatter(loadings['PC1'],loadings['PC2'])
axs[1].set_title('PC loadings plot')
axs[1].set_xlabel('PC1 %0.2f %%' % (pca.explained_variance_ratio_[0] * 100))
axs[1].set_ylabel('PC2 %0.2f %%' % (pca.explained_variance_ratio_[1] * 100))

axs[2].plot(mz,loadings['PC2'].abs())
# axs[2].plot(mz,loadings['PC3'])
axs[2].set_title('PC loadings plot')
axs[2].set_xlabel('m/z')
axs[2].set_ylabel('PC loadings');

## Model evaluation

In [ ]:
# Check group sizes

labelnames = set(file.iloc[:,-3])
labelnames = list(labelnames)

w = np.zeros(len(labelnames))
for i in range(0,len(labelnames)):
    group=file[file.iloc[:,-3]== labelnames[i]]
    w[i]=(len(group))
    print('group' + str(i) + ' sample size: '+str(w[i])+'\n')

print('ratio: '+str(w/max(w)))
weights = {labelnames[0]:1.0, labelnames[1]:w[0]/w[1]}

In [ ]:
X = Data_log
y = file['health']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
clf = LogisticRegressionCV(cv=5, random_state=0, solver='liblinear').fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
y_pred = pd.Series(clf.predict(X_test))
y_test = y_test.reset_index(drop=True)
z = pd.concat([y_test, y_pred], axis=1)
z.columns = ['True', 'Prediction']
z.head()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))

## Recursive feature selection

In [ ]:
# RFECV - use CV to determine & select best number of features
from sklearn.feature_selection import RFECV

model = LogisticRegression(solver='liblinear')

label = file['health'].tolist()
selector = RFECV(model, cv = 5, n_jobs = -1, step=5, scoring='accuracy', min_features_to_select = 30)
selector = selector.fit(Data_log, label)
features = selector.support_


In [ ]:
from collections import Counter

Counter(features).values()

In [ ]:
feature_names = Data_log.columns.values.tolist()

In [ ]:
feature_importance = list(zip(feature_names, selector.support_))
new_features = []
for key,value in enumerate(feature_importance):
    if(value[1]) == True:
        new_features.append(value[0])
        
print(new_features)

In [ ]:
import pickle

with open('/content/drive/MyDrive/selected_features_new2.pkl', 'wb') as f:
  pickle.dump(new_features, f)

## Selected logistic regression

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics

In [ ]:
selected_features = pickle.load( open( "drive/MyDrive/selected_features_new2.pkl", "rb" ) )

In [ ]:
new_X = Data_log[selected_features]

In [ ]:
X = new_X
y = file['health']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
clf = LogisticRegressionCV(cv=5, random_state=0, solver='liblinear').fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
y_pred = pd.Series(clf.predict(X_test))
y_test = y_test.reset_index(drop=True)
z = pd.concat([y_test, y_pred], axis=1)
z.columns = ['True', 'Prediction']
z.head()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))

In [ ]:
selected_features1 = pickle.load( open( "drive/MyDrive/selected_features.pkl", "rb" ) )
selected_features2 = pickle.load( open( "drive/MyDrive/selected_features_new2.pkl", "rb" ) )

In [ ]:
a = set(selected_features1).intersection(selected_features2)